In [12]:
import numpy as np 
import dynesty
import jax.numpy as jnp 
from jax import jit, grad

In [15]:
# Define the dimensionality of our problem.
ndim = 3

# Define our 3-D correlated multivariate normal log-likelihood.
C = np.identity(ndim)
C[C==0] = 0.95
Cinv = np.linalg.inv(C)
lnorm = -0.5 * (np.log(2 * np.pi) * ndim +
                np.log(np.linalg.det(C)))

@jit
def loglike(x):
    return -0.5 * jnp.dot(x, jnp.dot(Cinv, x)) + lnorm

# Define our uniform prior via the prior transform.
def ptform(u):
    return 20. * u - 10.

# Define our gradient with and without the Jacobian applied.
def grad_x(x):
    return -np.dot(Cinv, x)  # without Jacobian

def grad_u(x):
    return -np.dot(Cinv, x) * 20.  # with Jacobian for uniform [-10, 10)

def grad_j(x):
    return grad(loglike)(x) * 20

In [16]:
%%time

# Sample with `grad_u` (including Jacobian).
sampler = dynesty.NestedSampler(loglike, ptform, ndim, sample='rslice')

sampler.run_nested()
results_with_jac = sampler.results

iter: 3000 | bound: 7 | nc: 33 | ncall: 54935 | eff(%):  5.461 | loglstar:   -inf < -8.639 <    inf | logz: -16.377 +/-  0.116 | dlogz: 10.095 >  0.509                                               IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

iter: 4829 | +500 | bound: 16 | nc: 1 | ncall: 112839 | eff(%):  4.744 | loglstar:   -inf < -0.297 <    inf | logz: -9.170 +/-  0.129 | dlogz:  0.001 >  0.509                                        

CPU times: user 4.5 s, sys: 884 ms, total: 5.39 s
Wall time: 5.48 s


In [17]:
%%time

# Sample with `grad_u` (including Jacobian).
sampler = dynesty.NestedSampler(loglike, ptform, ndim, sample='hslice',
                                gradient=grad_j)
sampler.run_nested()
results_with_jac = sampler.results

iter: 4746 | +500 | bound: 36 | nc: 1 | ncall: 2701511 | eff(%):  0.194 | loglstar:   -inf < -0.306 <    inf | logz: -9.019 +/-  0.128 | dlogz:  0.001 >  0.509                                       

CPU times: user 3min 56s, sys: 1.63 s, total: 3min 58s
Wall time: 3min 58s


In [9]:
%%time

# Sample with `grad_x` (compute Jacobian numerically).
sampler = dynesty.NestedSampler(loglike, ptform, ndim, sample='hslice',
                                gradient=grad_x, compute_jac=True)
sampler.run_nested()
results_without_jac = sampler.results

iter: 4674 | +500 | bound: 36 | nc: 1 | ncall: 2663332 | eff(%):  0.194 | loglstar:   -inf < -0.309 <    inf | logz: -8.878 +/-  0.127 | dlogz:  0.001 >  0.509                                       

CPU times: user 36.6 s, sys: 4.6 s, total: 41.2 s
Wall time: 38.6 s
